In [8]:
!pip install opencv-python tensorflow keras matplotlib numpy pandas

In [9]:
import kagglehub
path = kagglehub.dataset_download("andrewmvd/face-mask-detection")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-detection


In [10]:
import os
import shutil
import xml.etree.ElementTree as ET
from tqdm import tqdm

images_path = os.path.join(path, "images")
annotations_path = os.path.join(path, "annotations")
output_path = "/content/data"

# folders for binary classification
os.makedirs(os.path.join(output_path, "with_mask"), exist_ok=True)
os.makedirs(os.path.join(output_path, "without_mask"), exist_ok=True)

for xml_file in tqdm(os.listdir(annotations_path)):
    tree = ET.parse(os.path.join(annotations_path, xml_file))
    root = tree.getroot()

    image_name = root.find("filename").text
    label = root.find("object").find("name").text

    if label == "mask_weared_incorrect":
        continue

    src = os.path.join(images_path, image_name)
    dest = os.path.join(output_path, label)

    if os.path.exists(src):
        shutil.copy(src, dest)


100%|██████████| 853/853 [00:02<00:00, 399.22it/s]


In [11]:
from sklearn.model_selection import train_test_split

# train/test folders
train_dir = "/content/final_data/train"
test_dir = "/content/final_data/test"

os.makedirs(train_dir + "/with_mask", exist_ok=True)
os.makedirs(train_dir + "/without_mask", exist_ok=True)
os.makedirs(test_dir + "/with_mask", exist_ok=True)
os.makedirs(test_dir + "/without_mask", exist_ok=True)

for category in ["with_mask", "without_mask"]:
    images = os.listdir(os.path.join(output_path, category))
    train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(output_path, category, img), os.path.join(train_dir, category))
    for img in test_imgs:
        shutil.copy(os.path.join(output_path, category, img), os.path.join(test_dir, category))


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(img_height, img_width),
                                               batch_size=batch_size,
                                               class_mode='binary')

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=(img_height, img_width),
                                             batch_size=batch_size,
                                             class_mode='binary')


Found 653 images belonging to 2 classes.
Found 164 images belonging to 2 classes.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')


In [14]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_data,
                    epochs=10,
                    validation_data=test_data)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.7682 - loss: 0.6122 - val_accuracy: 0.8537 - val_loss: 0.4292
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8614 - loss: 0.4230 - val_accuracy: 0.8537 - val_loss: 0.4444
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8686 - loss: 0.4523 - val_accuracy: 0.8537 - val_loss: 0.4287
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8577 - loss: 0.4205 - val_accuracy: 0.8537 - val_loss: 0.4524
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8392 - loss: 0.4582 - val_accuracy: 0.8537 - val_loss: 0.4246
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.8715 - loss: 0.3926 - val_accuracy: 0.8537 - val_loss: 0.4237
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.8370 - loss: 0.4270 - val_accuracy: 0.8537 - val_loss: 0.4263
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.8516 - loss: 0.4178 - val_accuracy: 0.8537 - val_loss:

In [15]:
loss, accuracy = model.evaluate(test_data)
print(f"Final Test Accuracy: {accuracy*100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 785ms/step - accuracy: 0.8679 - loss: 0.4000
Final Test Accuracy: 85.37%
